In [ ]:
from sympy import *
from sympy.matrices import BlockMatrix, Matrix, zeros
import sympy as sp
import numpy as np

In [ ]:
Ts = 0.09
dt = 0.01
foot_height_keep_STF = 0.2
N = int(Ts/dt)
print('Nb steps = ', N)
nb_total_variables_per_coordinate = int(3 * N) # px, vx, zx
print('nb_total_variables_per_coordinate', nb_total_variables_per_coordinate)
nb_total_variables = 3 * nb_total_variables_per_coordinate # x, y, z
print(nb_total_variables, 'variables')

In [ ]:
variables = []
coord = ['x', 'y', 'z']
for c in coord:
    for i in range(N):
        p = symbols(f'p{c}{i}')
        v = symbols(f'v{c}{i}')
        a = symbols(f'a{c}{i}')
        variables.extend([p, v, a])
pva_vector = Matrix(variables)
print('------------------- Optimization Variable -------------------')
print('pva_vector = ', len(pva_vector), ' variables')
for i in range(3):
    pprint(pva_vector.T[i*nb_total_variables_per_coordinate:(i+1)*nb_total_variables_per_coordinate])


In [ ]:
print('------------------- P Mat -------------------')
opt_weight_desired_pos = 10
opt_weight_desired_vel = 1
zero_matrix_3_by_3 = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

p_N_des = sp.Matrix([0.2, 0.3, 0.1])
v_N_des = sp.zeros(3)

q = zeros(nb_total_variables, 1)
P_matrix = zeros(nb_total_variables, nb_total_variables)

for i in range(nb_total_variables_per_coordinate):
    P_matrix[i * 3 + 2, i * 3 + 2] = 1.0

for i in range(3):
    P_matrix[(i + 1) * nb_total_variables_per_coordinate - 3, (i + 1) * nb_total_variables_per_coordinate - 3] = opt_weight_desired_pos
    P_matrix[(i + 1) * nb_total_variables_per_coordinate - 2, (i + 1) * nb_total_variables_per_coordinate - 2] = opt_weight_desired_vel

    q[(i+1) * nb_total_variables_per_coordinate - 3] = -2 * opt_weight_desired_pos * p_N_des[i]
    q[(i+1) * nb_total_variables_per_coordinate - 2] = -2 * opt_weight_desired_vel * v_N_des[i]

vPv = pva_vector.T * P_matrix * pva_vector

objective_function = vPv + q.T * pva_vector
print('Objective function')
pprint(objective_function)


In [ ]:

A_eq_pos_vel_des = sp.zeros(6, nb_total_variables)

A_eq_pos_vel_des[0, 0] = 1
A_eq_pos_vel_des[1, 1] = 1

A_eq_pos_vel_des[2, nb_total_variables_per_coordinate] = 1
A_eq_pos_vel_des[3, nb_total_variables_per_coordinate + 1] = 1

A_eq_pos_vel_des[4, 2*nb_total_variables_per_coordinate] = 1
A_eq_pos_vel_des[5, 2*nb_total_variables_per_coordinate + 1] = 1

result_vector = A_eq_pos_vel_des*pva_vector
print('------------------- Boundary Points Equality Contraints -------------------')
print('A_eq_pos_vel_desired * v = ')
pprint(result_vector)

In [ ]:
dt_symbol = symbols('dt')
block_dynamics = zeros(2, 6)
block_dynamics[0, 0] = 1
block_dynamics[0, 1] = dt_symbol
block_dynamics[0, 3] = -1
block_dynamics[1, 1] = 1
block_dynamics[1, 2] = dt_symbol
block_dynamics[1, 4] = -1
A_dynamics_per_coordinate = zeros(2 * N - 2, 3 * N)

j = 0
for i in range(int((2 * N - 2)/2)):
    A_dynamics_per_coordinate[j : j + 2, i * 3: i * 3 + 6] = block_dynamics
    j = j + 2

pva_vector_x_comp = pva_vector[0:nb_total_variables_per_coordinate, 0]

size_rows_A = A_dynamics_per_coordinate.shape[0]
size_cols_A = A_dynamics_per_coordinate.shape[1]
A_dynamics = zeros(3 * size_rows_A, nb_total_variables)

for i in range(3):
    A_dynamics[i * size_rows_A : i * size_rows_A + size_rows_A, i * size_cols_A : i * size_cols_A + size_cols_A] = A_dynamics_per_coordinate

mult = A_dynamics * pva_vector
print('------------------- Dynamic Contraints -------------------')
pprint(mult)

In [ ]:
block = zeros(2, 3)
block[0, 1] = 1
block[1, 2] = 1
A_limits = zeros(nb_total_variables_per_coordinate * 2, nb_total_variables)

for i in range(nb_total_variables_per_coordinate):
    A_limits[2*i:2*(i+1), 3*i:3*(i+1)] = block

pprint((A_limits*pva_vector).T)

In [ ]:
A_total = Matrix.vstack(A_eq_pos_vel_des,
                    A_dynamics,
                    A_limits)

print('A * PVA_VECTOR')
pprint(A_total*pva_vector)

In [ ]:
# Limits
p0_desired = 0.0
v0_desired = 0.0

l_boundary_pts_per_coordinate = Matrix([p0_desired, v0_desired])
u_boundary_pts_per_coordinate = Matrix([p0_desired, v0_desired])


l_boundary_pts = Matrix.vstack(l_boundary_pts_per_coordinate, l_boundary_pts_per_coordinate, l_boundary_pts_per_coordinate)
u_boundary_pts = Matrix.vstack(u_boundary_pts_per_coordinate, u_boundary_pts_per_coordinate, u_boundary_pts_per_coordinate)

l_boundary_pts

In [ ]:
v_max = 20.0
a_max = 100.0
l_limits_per_coordinate = zeros(2 * N, 1)
u_limits_per_coordinate = zeros(2 * N, 1)

for i in range(0, 2 * N, 2):
    l_limits_per_coordinate[i, 0] = -v_max
    u_limits_per_coordinate[i, 0] = v_max

for i in range(1, 2 * N, 2):
    l_limits_per_coordinate[i, 0] = -a_max
    u_limits_per_coordinate[i, 0] = a_max


l_limits = Matrix.vstack(l_limits_per_coordinate, l_limits_per_coordinate, l_limits_per_coordinate)
u_limits = Matrix.vstack(u_limits_per_coordinate, u_limits_per_coordinate, u_limits_per_coordinate)
pprint(l_limits.T)
pprint(u_limits.T)


In [ ]:

T_keep = 33/100*Ts
T_start_keep = 33/100*Ts
T_end_keep = T_keep + T_start_keep
n_keep = int(T_keep/dt)
n_start_keep = int(T_start_keep/dt)
n_end_keep = n_keep + n_start_keep

A_keep_foot = zeros(n_keep, nb_total_variables)

j = n_start_keep
for i in range(n_keep):
    A_keep_foot[i, 2*nb_total_variables_per_coordinate + 3*j] = 1
    j = j + 1

pprint(A_keep_foot @ pva_vector)

In [ ]:
pprint(pva_vector[0])
pprint(pva_vector[nb_total_variables_per_coordinate-3])

pprint(pva_vector[0:nb_total_variables_per_coordinate:3])

In [ ]:
import pandas as pd

df = pd.read_csv('output.csv')
df.head()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 3, figsize=(15, 10))
axs[0, 0].set_title('x')
axs[0, 0].plot(df['Position_X'], '.-r', label='pos')
axs[0, 0].set_ylabel('x [m]')
axs[1, 0].plot(df['Velocity_X'], '.-g', label='vel')
axs[1, 0].set_ylabel('v [m/s]')
# axs[1].plot(t[-1], v_N_desired, 'o')
axs[2, 0].plot(df['Acceleration_X'], '.-b', label='acc')
axs[2, 0].set_ylabel('a [m/s^2]')
axs[2, 0].set_xlabel('t [s]')

axs[0, 1].set_title('y')
axs[0, 1].plot(df['Position_Y'], '.-r', label='pos')
axs[1, 1].plot(df['Velocity_Y'], '.-g', label='vel')
# axs[1].plot(t[-1], v_N_desired, 'o')
axs[2, 1].plot(df['Acceleration_Y'], '.-b', label='acc')
axs[2, 1].set_xlabel('t [s]')

axs[0, 2].set_title('z')
axs[0, 2].plot(df['Position_Z'], '.-r', label='pos')
axs[1, 2].plot(df['Velocity_Z'], '.-g', label='vel')
axs[2, 2].plot(df['Acceleration_Z'], '.-b', label='acc')
axs[2, 2].set_xlabel('t [s]')

plt.tight_layout()
# [a.legend() for a in axs]
plt.show()